In [1]:
import pandas as pd 
#from bs4 import BeautifulSoup
import requests
from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager  
from tqdm import tqdm
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select  
from time import sleep  
pd.set_option('display.max_columns', None)

## 1. Metemos los csv de las pelis

In [2]:
data = pd.read_csv("test_misterio.csv", index_col=0)  #lo único que tenemos que cambiar cada vez es el nombre del csv
data.head()

,type,name,year,month,ids
genre,,,,,
Mystery,Movie,Spy Story,1990,3,tt0075259
Mystery,Movie,Demonia,1990,10,tt0094997
Mystery,Movie,Still Life,1990,11,tt0096177
Mystery,Movie,In the Cold of the Night,1990,12,tt0097573
Mystery,Movie,The Platinum Triangle,1990,3,tt0098100


In [3]:
ids = data["ids"] ##Esto no lo tocamos, ya está definido para coger únicamente el id de la peli que se va a leer en el buscador de IMDB
ids.head()

genre
Mystery    tt0075259
Mystery    tt0094997
Mystery    tt0096177
Mystery    tt0097573
Mystery    tt0098100
Name: ids, dtype: object

## 2. Extracción de los actores de las películas y guardamos la info en un csv

In [ ]:
driver = webdriver.Chrome()

driver.maximize_window() #Maximizamos ventana antes de meternos en IMDB

sleep(2)

driver.get("https://www.imdb.com/")

sleep(2)
#____________________________________________ PRIMERA PARTE: Extracción de nombre de actores____________________________________________________________________________________________

lista_actors = []

for id in tqdm(ids):

    driver.find_element("css selector", "#suggestion-search").send_keys(id, Keys.ENTER) #Para que coja el ID de cada tupla de peli

    lista_actores_consumible = []
    
    for i in range (1,11):

        try:
            nombre_actor = driver.find_element("css selector", f"#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > div > section > div > div.sc-9178d6fe-1.kFxVZc.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.gmonkL.title-cast.title-cast--movie.celwidget > div.ipc-shoveler.ipc-shoveler--base.ipc-shoveler--page0.title-cast__grid > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child({i}) > div.sc-bfec09a1-7.dpBDvu > a").text
            lista_actores_consumible.append (nombre_actor)

        except:
            lista_actores_consumible.append("NA")
    
    lista_actores_consumible.append(id) #Lo pongo al nivel del "for" para que lo apendee una vez y no cada vez que itere un actor y lo añado a la lista de consumible para que aparezca dentro de la lista de tuplas de actores y tb lo pongo antes de convertir la lista a tupla (línea de código a continuación)

    lista_actors.append (tuple(lista_actores_consumible))

    driver.back()

    sleep(2)

print (lista_actors)

In [ ]:
len(lista_actors)  #debe ser igual al nro de pelis en csv

In [ ]:
df_lista_actors = pd.DataFrame(lista_actors, columns=["actor1", "actor2", "actor3", "actor4", "actor5", "actor6", "actor7", "actor8", "actor9", "actor10","id"]) #convertimos "lista_actors" a dataframe

In [ ]:
df_lista_actors.to_csv("XXX_lista_actors.csv") #para guardarlo en un csv

## 3. Extracción de la información de los actores y guardamos la info en un csv

In [11]:
driver = webdriver.Chrome()

driver.maximize_window() #Maximizamos ventana antes de meternos en IMDB

sleep(2)

driver.get("https://www.imdb.com/")

sleep(2)

info_actores = []

for actores_peli in tqdm(lista_actors): 

    info_actores_consumible = []

    for actores in actores_peli[0:10]:
    
        info_actores_consumible.append (actores.upper()) #AÑADIMOS NOMBRE en mayúsculas para verlo bien

        if actores == "NA":
            info_actores_consumible.append ("NA nacimiento")
            info_actores_consumible.append ("NA conocido")
            info_actores_consumible.append ("NA profesión")
            info_actores_consumible.append ("NA oscars")


        else:

            driver.find_element("css selector", "#suggestion-search").send_keys(actores, Keys.ENTER) #busco cada actor en el "search box"

            sleep (2)
                    
            #Click en el resultado el actor para ver su información
            driver.find_element("css selector", "#__next > main > div.ipc-page-content-container.ipc-page-content-container--full.sc-383f2ac5-0.bfcGjo > div.ipc-page-content-container.ipc-page-content-container--center > section > div > div.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(3) > div.sc-17bafbdb-2.ffAEHI > ul > li:nth-child(1) > div.ipc-metadata-list-summary-item__c > div > a").click()
            
            sleep (2)

            try:
            
            #AÑADIMOS AÑO DE NACIMIENTO      
                try:
                    nacimiento = driver.find_element ("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > section > div:nth-child(4) > section > section > div.sc-e226b0e3-4.ecgcFy > div.sc-e226b0e3-6.hfusNC > div.sc-e226b0e3-11.jSQoAO > section > aside > div > span:nth-child(2)").text
                    try:
                        info_actores_consumible.append (nacimiento.split(",")[1]) 

                    except:
                        info_actores_consumible.append (nacimiento) 

                except:
                    info_actores_consumible.append ("NA nacimiento")
                
                sleep(2) 

                
                #AÑADIMOS POR QUÉ ES CONOCIDO           
                try:
                    conocido_lista = []
                    for i in range (1,5):
                        try:
                            conocido = driver.find_element("css selector", f"#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > div > section > div > div.sc-9178d6fe-1.kFxVZc.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.celwidget > section:nth-child(1) > div.sc-a6d4b6c0-0.jGufEe > div > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child({i}) > div.ipc-primary-image-list-card__content > div.ipc-primary-image-list-card__content-top")
                            conocido_limpio = conocido.text
                            conocido_lista.append (conocido_limpio)
                            sleep(3)

                        except NoSuchElementException:
                            continue

                    info_actores_consumible.append(conocido_lista)  

                except:
                    pass

                sleep(2)   


                #AÑADIMOS PROFESIÓN
                try:
                    profesion = driver.find_element ("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > section > div:nth-child(4) > section > section > div.sc-e226b0e3-3.jJsEuz > div > ul"). text
                    info_actores_consumible.append (profesion.split("\n"))

                
                except:
                    info_actores_consumible.append ("NA profesion")

                #AÑADIMOS INFO SOBRE OSCARS
                try:
                    oscars = driver.find_element ("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > div > section > div > div.sc-9178d6fe-1.kFxVZc.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(3) > div > ul > li > a.ipc-metadata-list-item__label.ipc-metadata-list-item__label--link").text
                    info_actores_consumible.append (oscars)           
                
                except:
                    info_actores_consumible.append ("No ha ganado ningún oscar")


            
            except:
                continue

    info_actores_consumible.append(actores_peli[10]) #Lo pongo al nivel del "for" para que lo apendee una vez y no cada vez que itere un actor y lo añado a la lista de consumible para que aparezca dentro de la lista de tuplas de actores y tb lo pongo antes de convertir la lista a tupla (línea de código a continuación)
    
    info_actores.append (tuple(info_actores_consumible))
    
    print (info_actores_consumible) #Para ir viendo que pelis se van descargando

    driver.back()

driver.close()  

print (info_actores)

100%|██████████| 10/10 [02:46<00:00, 16.65s/it]


[('Michael Petrovitch', 'Philip Latham', 'Don Fellows', 'Michael Gwynn', 'Nicholas Parsons', 'Toby Robins', 'Tessa Wyatt', 'Derren Nesbitt', 'Nigel Plaskitt', 'Ciaran Madden', 'tt0075259'), ('Brett Halsey', 'Meg Register', 'Lino Salemme', 'Christina Engelhardt', 'Pascal Druant', 'Grady Clarkson', 'Ettore Comi', 'Carla Cassola', 'Michael Aronin', 'Al Cliver', 'tt0094997'), ('Jason Gedrick', 'Jessica Steen', 'Stephen Shellen', 'Gary Farmer', 'Sam Malkin', 'Lubomir Mykytiuk', 'Dave McKay', 'Johnie Chase', 'Ted Loviscek', 'Beverly Murray', 'tt0096177'), ('Jeff Lester', 'Adrianne Sachs', 'Marc Singer', 'Brian Thompson', 'Shannon Tweed', 'John Beck', 'Tippi Hedren', 'David Soul', 'Jack Kehler', 'Stuart Bird', 'tt0097573'), ('Peter Lochran', 'Alex Courtney', 'Lori Lethin', 'Janet Carroll', 'Robert Frank Telfer', 'Joe Estevez', 'Troy Donahue', 'Harvey Jason', 'Jennifer Bassey', 'Ginger Pollini', 'tt0098100'), ('Héctor Bonilla', 'Héctor Ortega', 'María Rojo', 'Gina Morett', 'Pedro Armendáriz Jr

  0%|          | 0/10 [00:00<?, ?it/s]

['MICHAEL PETROVITCH', '1945', ['The Zoo Gang', 'Turkey Shoot', 'Spy Story', 'Poldark'], ['Actor'], 'No ha ganado ningún oscar', 'PHILIP LATHAM', ' 1929', ['Dracula, Prince of Darkness', 'The Treasure Seekers', 'The Fourth Arm', 'Middlemarch'], ['Actor'], 'No ha ganado ningún oscar', 'DON FELLOWS', ' 1922', ['Raiders of the Lost Ark', 'Superman IV: The Quest for Peace', 'Superman II', 'Lillie'], ['Actor'], 'No ha ganado ningún oscar', 'MICHAEL GWYNN', ' 1916', ['Jason and the Argonauts', 'Cleopatra', 'Village of the Damned', 'Great Expectations'], ['Actor'], 'No ha ganado ningún oscar', 'NICHOLAS PARSONS', ' 1923', ['Doctor Who', 'Happy Is the Bride', 'Saturday Playhouse', 'Lady Godiva'], ['Actor', 'Director', 'Writer'], 'Awards', 'TOBY ROBINS', 'NA nacimiento', ['For Your Eyes Only', 'Space: 1999', 'Folio', 'Department S'], ['Actress'], 'No ha ganado ningún oscar', 'TESSA WYATT', 'NA nacimiento', ['Pride and Prejudice', 'The Wednesday Play', 'Armchair Theatre', 'BBC Play of the Month'

 10%|█         | 1/10 [04:11<37:40, 251.22s/it]

...

 30%|███       | 3/10 [12:36<29:28, 252.65s/it]

['JEFF LESTER', ' 1955', ['Star Trek IV: The Voyage Home', 'The Little Drummer Girl', 'The Last Real Cowboys', 'Speed of Life'], ['Actor', 'Writer', 'Director'], 'Awards', 'ADRIANNE SACHS', 'NA nacimiento', ['RoboCop', 'The Stuff', 'Best of the Best', 'Cat Chaser'], ['Actress'], 'No ha ganado ningún oscar', 'MARC SINGER', ' 1948', ['The Beastmaster', 'V', 'Beastmaster 2: Through the Portal of Time', 'V'], ['Actor', 'Director', 'Additional Crew'], 'No ha ganado ningún oscar', 'BRIAN THOMPSON', ' 1959', ['Cobra', 'The Terminator', 'Mortal Kombat: Annihilation', 'DragonHeart'], ['Actor', 'Director', 'Writer'], 'Awards', 'SHANNON TWEED', ' 1957', ['Electra', 'Detroit Rock City', 'Cannibal Women in the Avocado Jungle of Death', 'No Contest'], ['Actress', 'Producer', 'Additional Crew'], 'No ha ganado ningún oscar', 'JOHN BECK', ' 1943', ['Rollerball', 'Sleeper', 'Black Day Blue Night', 'Audrey Rose'], ['Actor', 'Soundtrack'], 'No ha ganado ningún oscar', 'TIPPI HEDREN', ' 1930', ['The Birds'

 40%|████      | 4/10 [17:03<25:48, 258.07s/it]

['PETER LOCHRAN', 'NA nacimiento', ['Capitol', 'The Young Doctors', 'The Platinum Triangle', 'Another World'], ['Actor'], 'Awards', 'ALEX COURTNEY', 'March 21', ['Mr. Goodbar', 'Sword of Justice', 'And the Band Played On', 'Murder Between Friends'], ['Actor'], 'No ha ganado ningún oscar', 'LORI LETHIN', 'NA nacimiento', ['The Day After', 'Brokedown Palace', 'Bloody Birthday', 'Return to Horror High'], ['Actress'], 'No ha ganado ningún oscar', 'JANET CARROLL', ' 1940', ['Risky Business', 'Enough', 'Forces of Nature', 'Family Business'], ['Actress', 'Soundtrack'], 'No ha ganado ningún oscar', 'ROBERT FRANK TELFER', 'NA nacimiento', ['Arachnophobia', 'Mr. Deeds', "Something's Gotta Give", 'As the World Turns'], ['Actor'], 'No ha ganado ningún oscar', 'JOE ESTEVEZ', ' 1946', ['Return of the Roller Blade Seven', 'Who Lives Last', 'The Roller Blade Seven', 'Eyes Upon Waking'], ['Actor', 'Producer', 'Additional Crew'], 'Awards', 'TROY DONAHUE', ' 1936', ['A Summer Place', 'Cry-Baby', 'The God

 50%|█████     | 5/10 [21:17<21:24, 256.83s/it]

In [ ]:
len(info_actores)  #debe ser igual al nro de pelis en csv

In [ ]:
df_info_actores = pd.DataFrame(info_actores) #convertimos "info_actors" a dataframe

In [ ]:
df_info_actores.to_csv("XXX_info_actores.csv") #para guardarlo en un csv